# STARTING FOR DTM. Obtaining NDVI of only soil cover (0.07-0.10)

In [2]:
import arcpy
import os

# Set the input and output folder paths
input_folder = r"C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT"  # Update this path
output_folder = r"C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\SOIL_MASK"  # Update this path

# Path to the shapefile for clipping
clip_shapefile = r"C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\OLA.shp"  # Update this path

# Ensure the output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Get a list of all TIFF files in the input folder
tif_files = [f for f in os.listdir(input_folder) if f.endswith('.tif')]

# Loop through each TIFF file and apply the NDVI calculation
for tif_file in tif_files:
    # Define the full path to the input TIFF file
    input_path = os.path.join(input_folder, tif_file)
    
    # Define output file path with "_soilmask" suffix
    output_filename = tif_file.replace('.tif', '_soilmask.tif')
    output_path = os.path.join(output_folder, output_filename)

    # Read the raster bands
    red_band = arcpy.Raster(os.path.join(input_path + r"\Band_3"))  # Band 3 is Red
    nir_band = arcpy.Raster(os.path.join(input_path + r"\Band_5"))  # Band 5 is NIR

    # Calculate NDVI: (NIR - Red) / (NIR + Red)
    ndvi = (nir_band - red_band) / (nir_band + red_band)

    # Apply conditions: Set NDVI to 1 if it's between 0.1 and 0.15, otherwise set it to NoData (null)
    ndvi_soilmask = arcpy.sa.Con((ndvi >= 0.07) & (ndvi <= 0.10), 1, arcpy.sa.SetNull(ndvi, ndvi))

    # Clip the NDVI soil mask raster using the shapefile
    clipped_raster = arcpy.management.Clip(ndvi_soilmask, "#", output_path, clip_shapefile, "NoData", "ClippingGeometry", "MAINTAIN_EXTENT")

    # Save the processed and clipped NDVI soil mask raster to the output path
    # arcpy.management.Clip already writes the result to the output_path, no need for a separate .save() call.
    
    print(f"Processed and saved: {output_path}")

print("NDVI soil mask processing complete.")


Processed and saved: C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\SOIL_MASK\RIP_20220707_1124_6band_soilmask.tif
Processed and saved: C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\SOIL_MASK\RIP_20220707_1249_6band_soilmask.tif
Processed and saved: C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\SOIL_MASK\RIP_20220707_1512_6band_soilmask.tif
Processed and saved: C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\SOIL_MASK\RIP_20220708_1119_6band_soilmask.tif
Processed and saved: C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\SOIL_MASK\RIP_20220708_1252_6band_soilmask.tif
Processed and saved: C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\SOIL_MASK\RIP_20220708_1514_6band_soilmask.tif
NDVI soil mask processing complete.


# DSM SOIL (soil mask x DSM)

In [3]:
import arcpy
import os

# Set the input and output folder paths
input_folder1 = r"C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\SOIL_MASK"  # Update this path
input_folder2 = r"C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\DSM"  # Update this path
output_folder = r"C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DSM_SOIL"  # Update this path

# Ensure the output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Get a list of all TIFF files in both input folders
tif_files1 = [f for f in os.listdir(input_folder1) if f.endswith('.tif')]
tif_files2 = [f for f in os.listdir(input_folder2) if f.endswith('.tif')]

# Create a dictionary to map files from input_folder2 by the first 17 characters
file_dict2 = {f[:17]: f for f in tif_files2}

# Loop through each TIFF file in the first folder
for tif_file1 in tif_files1:
    # Extract the first 17 characters to match correlated files
    file_key = tif_file1[:17]
    
    # Check if there's a corresponding file in the second folder
    if file_key in file_dict2:
        # Define the full paths to the input TIFF files
        input_path1 = os.path.join(input_folder1, tif_file1)
        input_path2 = os.path.join(input_folder2, file_dict2[file_key])
        
        # Define the output file path with "_dsm_soil" suffix
        output_filename = tif_file1.replace('.tif', '_dsm_soil.tif')
        output_path = os.path.join(output_folder, output_filename)

        # Read the raster files
        raster1 = arcpy.Raster(input_path1)
        raster2 = arcpy.Raster(input_path2)

        # Multiply the rasters
        multiplied_raster = raster1 * raster2

        # Save the multiplied raster to the output path
        multiplied_raster.save(output_path)

        print(f"Processed and saved: {output_path}")

print("Processing complete.")


Processed and saved: C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DSM_SOIL\RIP_20220707_1124_6band_soilmask_dsm_soil.tif
Processed and saved: C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DSM_SOIL\RIP_20220707_1249_6band_soilmask_dsm_soil.tif
Processed and saved: C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DSM_SOIL\RIP_20220707_1512_6band_soilmask_dsm_soil.tif
Processed and saved: C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DSM_SOIL\RIP_20220708_1119_6band_soilmask_dsm_soil.tif
Processed and saved: C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DSM_SOIL\RIP_20220708_1252_6band_soilmask_dsm_soil.tif
Processed and saved: C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DSM_SOIL\RIP_20220708_1514_6band_soilmask_dsm_soil.tif
Processing complete.


# DTM KRIGING

In [5]:
# Import arcpy module
import arcpy
from arcpy.sa import *
import os

# Set environment settings
arcpy.env.workspace = r"C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DSM_SOIL"  # Replace with your input folder path
arcpy.env.overwriteOutput = True

# Specify the output folder for the point shapefiles and Kriging rasters
output_folder = r"C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DTM_KRIG"  # Replace with your desired output folder path

# List all TIFF files in the input folder
tiff_files = arcpy.ListRasters("*.tif")

# Loop through each TIFF file in the folder
for tiff_file in tiff_files:
    print(f"Processing {tiff_file}...")

    # Step 1: Exclude zero and NoData values
    valid_raster = Con(Raster(tiff_file) > 0, Raster(tiff_file))

    # Set environment settings for random raster
    raster_extent = arcpy.Describe(tiff_file).extent
    cell_size = arcpy.Describe(tiff_file).meanCellHeight  # or meanCellWidth if needed

    # Step 2: Create a random raster with values between 0 and 1
    random_raster = arcpy.management.CreateRandomRaster(
        out_path=arcpy.env.workspace,
        out_name=f"random_{os.path.basename(tiff_file)}",
        distribution="Uniform",
        raster_extent=raster_extent,
        cellsize=cell_size
    )

    # Step 3: Select 10% of random pixels
    random_raster_layer = Raster(random_raster)
    selected_raster = Con(random_raster_layer <= 0.1, valid_raster)

    # Step 4: Convert the selected raster to a point shapefile
    output_points = os.path.join(arcpy.env.workspace, f"{os.path.splitext(tiff_file)[0]}_soil_pts.shp")
    arcpy.RasterToPoint_conversion(selected_raster, output_points, "VALUE")
    print(f"Point shapefile saved to {output_points}")

    # Step 5: Define Kriging model parameters
    semivariogram_type = "Linear"  # Example semivariogram type, could be "Exponential", "Gaussian", etc.
    lag_size = 10  # Example lag size, adjust as needed
    major_range = 100  # Example range, adjust as needed
    partial_sill = 0.5  # Example partial sill, adjust as needed
    nugget = 0.1  # Example nugget, adjust as needed

    # Create the Kriging model
    kriging_model = KrigingModelOrdinary(
        semivariogram_type,
        lag_size,
        major_range,
        partial_sill,
        nugget
    )

    # Step 6: Perform Kriging on the point shapefile
    kriging_output = os.path.join(output_folder, f"{os.path.splitext(tiff_file)[0]}_dtm_krig.tif")

    # Perform Kriging interpolation
    kriging_result = Kriging(
        in_point_features=output_points,
        z_field="grid_code",  # Changed to 'grid_code'
        kriging_model=kriging_model,
        cell_size=6.5
    )

    # Save the Kriging output raster to the specified output folder
    kriging_result.save(kriging_output)
    print(f"Kriging raster saved to {kriging_output}")


Processing RIP_20220707_1124_6band_soilmask_dsm_soil.tif...
Point shapefile saved to C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DSM_SOIL\RIP_20220707_1124_6band_soilmask_dsm_soil_soil_pts.shp
Kriging raster saved to C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DTM_KRIG\RIP_20220707_1124_6band_soilmask_dsm_soil_dtm_krig.tif
Processing RIP_20220707_1249_6band_soilmask_dsm_soil.tif...
Point shapefile saved to C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DSM_SOIL\RIP_20220707_1249_6band_soilmask_dsm_soil_soil_pts.shp
Kriging raster saved to C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DTM_KRIG\RIP_20220707_1249_6band_soilmask_dsm_soil_dtm_krig.tif
Processing RIP_20220707_1512_6band_soilmask_dsm_soil.tif...
Point shapefile saved to C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DSM_SOIL\RIP_20220707_1512_6band_soilmask_dsm_soil_soil_pts.shp
Kriging rast

In [1]:
# Import arcpy module
import arcpy
from arcpy.sa import *
import os

# Set environment settings
arcpy.env.workspace = r"C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DSM_SOIL"  # Replace with your input folder path
arcpy.env.overwriteOutput = True

# Specify the output folder for the point shapefiles and Kriging rasters
output_folder = r"C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DTM_KRIG"  # Replace with your desired output folder path

# NAD 1983 (2011) StatePlane California III FIPS 0403 (Meters) projection
target_projection = arcpy.SpatialReference(6415)  # EPSG code for NAD 1983 (2011) StatePlane California III FIPS 0403 (Meters)

# List all TIFF files in the input folder
tiff_files = arcpy.ListRasters("*.tif")

# Loop through each TIFF file in the folder
for tiff_file in tiff_files:
    print(f"Processing {tiff_file}...")

    # Step 1: Exclude zero and NoData values
    valid_raster = Con(Raster(tiff_file) > 0, Raster(tiff_file))

    # Set environment settings for random raster
    raster_extent = arcpy.Describe(tiff_file).extent
    cell_size = arcpy.Describe(tiff_file).meanCellHeight  # or meanCellWidth if needed

    # Step 2: Create a random raster with values between 0 and 1
    random_raster = arcpy.management.CreateRandomRaster(
        out_path=arcpy.env.workspace,
        out_name=f"random_{os.path.basename(tiff_file)}",
        distribution="Uniform",
        raster_extent=raster_extent,
        cellsize=cell_size
    )

    # Step 3: Select 10% of random pixels
    random_raster_layer = Raster(random_raster)
    selected_raster = Con(random_raster_layer <= 0.1, valid_raster)

    # Step 4: Convert the selected raster to a point shapefile
    output_points = os.path.join(arcpy.env.workspace, f"{os.path.splitext(tiff_file)[0]}_soil_pts.shp")
    arcpy.RasterToPoint_conversion(selected_raster, output_points, "VALUE")
    print(f"Point shapefile saved to {output_points}")

    # Step 5: Define Kriging model parameters
    semivariogram_type = "Linear"  # Example semivariogram type, could be "Exponential", "Gaussian", etc.
    lag_size = 10  # Example lag size, adjust as needed
    major_range = 100  # Example range, adjust as needed
    partial_sill = 0.5  # Example partial sill, adjust as needed
    nugget = 0.1  # Example nugget, adjust as needed

    # Create the Kriging model
    kriging_model = KrigingModelOrdinary(
        semivariogram_type,
        lag_size,
        major_range,
        partial_sill,
        nugget
    )

    # Step 6: Perform Kriging on the point shapefile
    kriging_output = os.path.join(output_folder, f"{os.path.splitext(tiff_file)[0]}_dtm_krig.tif")

    # Perform Kriging interpolation
    kriging_result = Kriging(
        in_point_features=output_points,
        z_field="grid_code",  # Changed to 'grid_code'
        kriging_model=kriging_model,
        cell_size=6.5
    )

    # Save the Kriging output raster to the specified output folder
    kriging_result.save(kriging_output)
    print(f"Kriging raster saved to {kriging_output}")

    # Step 7: Project the Kriging output raster to NAD 1983 (2011) StatePlane California III FIPS 0403 (Meters)
    projected_output = os.path.join(output_folder, f"{os.path.splitext(tiff_file)[0]}_dtm_krig_projected.tif")
    arcpy.management.ProjectRaster(
        in_raster=kriging_output,
        out_raster=projected_output,
        out_coor_system=target_projection,
        resampling_type="BILINEAR"
    )
    print(f"Projected Kriging raster saved to {projected_output}")


Processing RIP_20220707_1124_6band_soilmask_dsm_soil.tif...
Point shapefile saved to C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DSM_SOIL\RIP_20220707_1124_6band_soilmask_dsm_soil_soil_pts.shp
Kriging raster saved to C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DTM_KRIG\RIP_20220707_1124_6band_soilmask_dsm_soil_dtm_krig.tif
Projected Kriging raster saved to C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DTM_KRIG\RIP_20220707_1124_6band_soilmask_dsm_soil_dtm_krig_projected.tif
Processing RIP_20220707_1249_6band_soilmask_dsm_soil.tif...
Point shapefile saved to C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DSM_SOIL\RIP_20220707_1249_6band_soilmask_dsm_soil_soil_pts.shp
Kriging raster saved to C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DTM_KRIG\RIP_20220707_1249_6band_soilmask_dsm_soil_dtm_krig.tif
Projected Kriging raster saved to C:\Users\14352\Desk

# HEIGHT

In [1]:
import arcpy
from arcpy.sa import *
import os

# Define input parameters
tif_folder1 = r"C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\DSM"  # Folder with first set of TIFF files
tif_folder2 = r"C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DTM_KRIG"  # Folder with second set of TIFF files
grid_shapefile = r"C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\OLA.shp"  # Grid shapefile
output_folder = r"C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\HEIGHT"  # Folder to save output files

# Set environment settings
arcpy.env.workspace = tif_folder1  # Set workspace to the first folder
arcpy.env.overwriteOutput = True  # Overwrite existing outputs

# Create a dictionary of files in tif_folder2 for easier matching
tif2_dict = {tif[:17]: tif for tif in os.listdir(tif_folder2) if tif.endswith('.tif')}

# Loop through each TIFF file in folder1
for tif1 in arcpy.ListRasters("*.tif"):
    base_name = tif1[:17]  # Extract first 17 characters to match between folders

    # Check if there's a matching file in folder2 based on the first 17 characters
    if base_name in tif2_dict:
        tif2 = os.path.join(tif_folder2, tif2_dict[base_name])  # Get full path of matching tif2
        print(f"Processing {tif1} and {tif2}...")

        # Perform raster subtraction: tif1 - tif2
        raster_diff = Raster(tif1) - Raster(tif2)

        # Apply conditional statement: if raster_diff < 0.1, set it to 0
        raster_conditional = Con(raster_diff < 0.1, 0, raster_diff)

        # Save the difference raster temporarily
        diff_output = f"{output_folder}\\{base_name}_diff.tif"
        raster_conditional.save(diff_output)

        # Perform zonal statistics as a table to get the 95th percentile for each grid cell
        zonal_stats_table = f"{output_folder}\\zonal_stats_{base_name}.dbf"
        
        # Using ZonalStatisticsAsTable with correct parameters
        arcpy.sa.ZonalStatisticsAsTable(
            in_zone_data=grid_shapefile,                # Grid shapefile
            zone_field="FID",                          # Field representing zone (FID)
            in_value_raster=diff_output,                # Raster to calculate statistics from
            out_table=zonal_stats_table,                # Output table
            ignore_nodata="DATA",                       # Ignore NoData values
            statistics_type="PERCENTILE",               # Type of statistics (percentile)
            percentile_values=[95]                      # Percentile to calculate (95th percentile)
        )
        
        # Join the zonal statistics table (FID_ from the stats table) back to the grid shapefile by FID
        arcpy.JoinField_management(grid_shapefile, "FID", zonal_stats_table, "FID_", ["PCT95"])

        # Convert the updated grid shapefile to points (to prepare for raster creation)
        point_output = f"{output_folder}\\points_{base_name}.shp"
        arcpy.FeatureToPoint_management(grid_shapefile, point_output, "INSIDE")

        # Convert the points to a raster, using the 95th percentile field ("PCT95")
        raster_output = f"{output_folder}\\{base_name}_height.tif"
        arcpy.PointToRaster_conversion(point_output, "PCT95", raster_output, "MEAN", "NONE", 6.5)

        print(f"Saved output raster: {raster_output}")

        # Delete the joined field (PCT95) from the shapefile after processing
        arcpy.DeleteField_management(grid_shapefile, ["PCT95"])

        # Optional: Clean up any temporary files if needed (like zonal stats table or points shapefile)

print("Processing completed!")


Processing RIP_20220707_1124_DSM.tif and C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DTM_KRIG\RIP_20220707_1124_6band_soilmask_dsm_soil_dtm_krig_projected.tif...
Saved output raster: C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\HEIGHT\RIP_20220707_1124_height.tif
Processing RIP_20220707_1249_DSM.tif and C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DTM_KRIG\RIP_20220707_1249_6band_soilmask_dsm_soil_dtm_krig_projected.tif...
Saved output raster: C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\HEIGHT\RIP_20220707_1249_height.tif
Processing RIP_20220707_1512_DSM.tif and C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\DTM_KRIG\RIP_20220707_1512_6band_soilmask_dsm_soil_dtm_krig_projected.tif...
Saved output raster: C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\HEIGHT\RIP_20220707_1512_height.tif
Processing RIP_20220708_1119_DSM.tif 

# WIDTH OVER HEIGHT

In [1]:
import arcpy
from arcpy.sa import *
import os

# Define input parameters
tif_folder1 = r"C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\WIDTH"  # Folder with first set of TIFF files
tif_folder2 = r"C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\HEIGHT"  # Folder with second set of TIFF files
output_folder = r"C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\WIDTH_OVER_HEIGHT"  # Folder to save output files

# Set environment settings
arcpy.env.workspace = tif_folder1  # Set workspace to the first folder
arcpy.env.overwriteOutput = True  # Overwrite existing outputs

# Create a dictionary of files in tif_folder2 for easier matching
tif2_dict = {tif[:17]: tif for tif in os.listdir(tif_folder2) if tif.endswith('.tif')}

# Loop through each TIFF file in folder1
for tif1 in arcpy.ListRasters("*.tif"):
    base_name = tif1[:17]  # Extract first 17 characters to match between folders

    # Check if there's a matching file in folder2 based on the first 17 characters
    if base_name in tif2_dict:
        tif2 = os.path.join(tif_folder2, tif2_dict[base_name])  # Get full path of matching tif2
        print(f"Processing {tif1} and {tif2}...")

        # Load the rasters
        raster1 = Raster(tif1)
        raster2 = Raster(tif2)

        # Apply conditional to handle division by zero: if tif2 is 0, the result is 0
        raster_division = Con(raster2 == 0, 0, raster1 / raster2)

        # Save the result with "_woverh" appended to the filename
        output_filename = f"{output_folder}\\{base_name}_woverh.tif"
        raster_division.save(output_filename)

        print(f"Saved output raster: {output_filename}")

print("Processing completed!")


Processing RIP_20220707_1124_6band_NDVI_canopy_FC_OK_width.tif and C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\HEIGHT\RIP_20220707_1124_height.tif...
Saved output raster: C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\WIDTH_OVER_HEIGHT\RIP_20220707_1124_woverh.tif
Processing RIP_20220707_1249_6band_NDVI_canopy_FC_OK_width.tif and C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\HEIGHT\RIP_20220707_1249_height.tif...
Saved output raster: C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\WIDTH_OVER_HEIGHT\RIP_20220707_1249_woverh.tif
Processing RIP_20220707_1512_6band_NDVI_canopy_FC_OK_width.tif and C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\HEIGHT\RIP_20220707_1512_height.tif...
Saved output raster: C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2022\MS-ALTUMPT\WIDTH_OVER_HEIGHT\RIP_20220707_1512_woverh.tif
Processing RIP_20220708_1119_6band_NDVI_